# Absenteeism - Busines Case Example - cont

## Create LogisticRegression

### Import libraries

In [1]:
import pandas as pd
import numpy as np

### Load data

In [2]:
data_preprocessed = pd.read_csv("Absenteeism_preprocessed.csv")
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


## Create the targets

In [3]:
# We will use the logistic regression as this is a classification problem. This regression is useful when we have 
# two targets. We will consider normal absenteeism (below the median) and extensive absenteeism (above the median).

In [4]:
data_preprocessed["Absenteeism Time in Hours"].median()

3.0

In [5]:
targets = np.where(data_preprocessed["Absenteeism Time in Hours"] > 
                   data_preprocessed["Absenteeism Time in Hours"].median(), 1, 0)

In [6]:
data_preprocessed["Excessive Absenteeism"] = targets
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


In [8]:
# Median is a good indicator because we could naturally balance the data: numbers of 0 and 1 are similar
targets.sum()/targets.shape[0]

0.45571428571428574

In [9]:
# We drop the targets, but also 3 other columns which we saw at the end that have no impact on the model
data_with_targets = data_preprocessed.drop(["Absenteeism Time in Hours", "Day of the week", 
                                            "Distance to Work", "Daily Work Load Average"], axis=1)

data_with_targets is data_preprocessed

False

## Select inputs

In [10]:
data_with_targets.shape

(700, 12)

In [11]:
unscaled_inputs = data_with_targets.iloc[:,:-1]

## Standardize the data

In [12]:
# This piece was given when we standardized dummies, but THEY SHOULD NOT BE STANDARDIZED!!! because this does not
# make a good predictive power.

In [13]:
# We need to specify which columns should be standardized and thus we make a class which will be choosing them
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.scaler = StandardScaler()
        self.columns = columns
        
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        return self
    
    def transform(self, X, y=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), 
                                columns=self.columns
                               )
        X_not_scaled = X.loc[:, ~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [14]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [17]:
# We choose all columns without dummies: reasons and education

# columns_to_scale = ['Month value', 'Transportation Expense',
                    #'Age', 'Body Mass Index', 'Children', 'Pets']
    
# A good way to define columns to scale is by first choosing columns to omit
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Education']

columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [18]:
absenteeism_scaler = CustomScaler(columns_to_scale)

In [19]:
# Use scaler to fit data
absenteeism_scaler.fit(unscaled_inputs)

# Use scaler to transform data
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.388293,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.388293,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.388293,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


## Split the data into train and test, and shuffle

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size=0.8, random_state=20)

In [23]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(560, 11) (140, 11) (560,) (140,)


## Logistic regression

We use here sklearn and not Statsmodels because sklearn is more stable for complex data.

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

## Training model

In [25]:
reg = LogisticRegression()

In [26]:
reg.fit(x_train, y_train)

LogisticRegression()

In [27]:
# To get a mean accuracy
reg.score(x_train, y_train)

0.7732142857142857

## Manually check the accuracy

In [28]:
# Finding predicted outputs of the regression
model_outputs = reg.predict(x_train)
model_outputs

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [29]:
y_train

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,

In [30]:
model_outputs == y_train
np.sum(model_outputs == y_train)/y_train.shape[0]

0.7732142857142857

## Finding coefficients and intercept

In [28]:
# We need a function for an external use (as for Tableu), and thus we need coefficients and weights:

In [31]:
reg.intercept_

array([-1.6474549])

In [32]:
reg.coef_

array([[ 2.80019733,  0.95188356,  3.11555338,  0.83900082,  0.1589299 ,
         0.60528415, -0.16989096,  0.27981088, -0.21053312,  0.34826214,
        -0.27739602]])

In [33]:
# To get input names we refer to the unscaled_inputs as this is dataframe which has names.
# Scaled_inputs are arrays, where names are not given.

feature_name = unscaled_inputs.columns.values

In [34]:
summary_table = pd.DataFrame(columns=["Feature Name"], data=feature_name)
summary_table["Coefficients"] = np.transpose(reg.coef_)
summary_table

,Feature Name,Coefficients
0,Reason_1,2.800197
1,Reason_2,0.951884
2,Reason_3,3.115553
3,Reason_4,0.839001
4,Month value,0.158930
5,Transportation Expense,0.605284
6,Age,-0.169891
7,Body Mass Index,0.279811
8,Education,-0.210533
9,Children,0.348262


In [35]:
# Add an intercept in the first row
summary_table.index = summary_table.index+1
summary_table.loc[0] = ["Intercept", reg.intercept_[0]]
summary_table = summary_table.sort_index()

In [36]:
summary_table

# The equation is: log(odds) = intercept + Reason_1*2.07 + .......

,Feature Name,Coefficients
0,Intercept,-1.647455
1,Reason_1,2.800197
2,Reason_2,0.951884
3,Reason_3,3.115553
4,Reason_4,0.839001
5,Month value,0.158930
6,Transportation Expense,0.605284
7,Age,-0.169891
8,Body Mass Index,0.279811
9,Education,-0.210533


In [37]:
summary_table["Odds Ratio"] = np.exp(summary_table["Coefficients"])

In [38]:
summary_table.sort_values("Odds Ratio", ascending=False)

,Feature Name,Coefficients,Odds Ratio
3,Reason_3,3.115553,22.545903
1,Reason_1,2.800197,16.447892
2,Reason_2,0.951884,2.590585
4,Reason_4,0.839001,2.314054
6,Transportation Expense,0.605284,1.831773
10,Children,0.348262,1.416604
8,Body Mass Index,0.279811,1.322880
5,Month value,0.158930,1.172256
7,Age,-0.169891,0.843757
9,Education,-0.210533,0.810152


If the coefficient is around 0 or odds ratio is around 1 we do not see any change for the corresponding 
input/output. Thus, this feature is useless for the model and can be ignored or just dropped.

If we see at this point that there are some useless coefficients we do backward elimination:
we go to the beggining and drop the 3 columns: "Daily Work Load Average", "Distance to Work", "Day of the week",
and then run the model again.

## Testing the model

In [40]:
reg.score(x_test, y_test)

0.75

If the test accuracy is 10-20% lower than the train accuracy it means that our model overfits data.

In [41]:
# Getting probability of our outputs
predicted_proba = reg.predict_proba(x_test)
predicted_proba

# The first column shows the probability of our observation being 0, and the second column is the probability of
# the observation being 1.

array([[0.71340413, 0.28659587],
       [0.58724228, 0.41275772],
       [0.44020821, 0.55979179],
       [0.78159464, 0.21840536],
       [0.08410854, 0.91589146],
       [0.33487603, 0.66512397],
       [0.29984576, 0.70015424],
       [0.13103971, 0.86896029],
       [0.78625404, 0.21374596],
       [0.74903632, 0.25096368],
       [0.49397598, 0.50602402],
       [0.22484913, 0.77515087],
       [0.07129151, 0.92870849],
       [0.73178133, 0.26821867],
       [0.30934135, 0.69065865],
       [0.5471671 , 0.4528329 ],
       [0.55052275, 0.44947725],
       [0.5392707 , 0.4607293 ],
       [0.40201117, 0.59798883],
       [0.05361575, 0.94638425],
       [0.7003009 , 0.2996991 ],
       [0.78159464, 0.21840536],
       [0.42037128, 0.57962872],
       [0.42037128, 0.57962872],
       [0.24783565, 0.75216435],
       [0.74566259, 0.25433741],
       [0.51017274, 0.48982726],
       [0.85690195, 0.14309805],
       [0.20349733, 0.79650267],
       [0.78159464, 0.21840536],
       [0.

In [42]:
# We are interested in extensive abseteeism, so the probability of getting one, so we extract this column
predicted_proba[:,1]

array([0.28659587, 0.41275772, 0.55979179, 0.21840536, 0.91589146,
       0.66512397, 0.70015424, 0.86896029, 0.21374596, 0.25096368,
       0.50602402, 0.77515087, 0.92870849, 0.26821867, 0.69065865,
       0.4528329 , 0.44947725, 0.4607293 , 0.59798883, 0.94638425,
       0.2996991 , 0.21840536, 0.57962872, 0.57962872, 0.75216435,
       0.25433741, 0.48982726, 0.14309805, 0.79650267, 0.21840536,
       0.36956558, 0.67906035, 0.68502567, 0.52868083, 0.21840536,
       0.53506551, 0.22147081, 0.73692105, 0.40498044, 0.60505988,
       0.21075848, 0.45224466, 0.23751292, 0.39833498, 0.82755447,
       0.56797575, 0.69113325, 0.28659587, 0.21935267, 0.2033097 ,
       0.57628256, 0.3294664 , 0.66512397, 0.26949499, 0.83321968,
       0.43491525, 0.88374612, 0.23127072, 0.33415858, 0.34432939,
       0.69909345, 0.65494263, 0.29244941, 0.79200758, 0.20750276,
       0.26840558, 0.08708566, 0.22147081, 0.73245417, 0.30530219,
       0.22147081, 0.29014408, 0.90438191, 0.46061297, 0.60174

## Save the model

In [43]:
# Saving the model is saving the "reg" object as it contains all info that we need.

# We will use "pickle" which converts Python object into a character string.
import pickle

with open("model", "wb") as file:
    pickle.dump(reg, file)

In [44]:
# We also need to save the scaler, as it can be used to train other data.
with open("scaler", "wb") as file_scaler:
    pickle.dump(absenteeism_scaler, file_scaler)

In [45]:
# To use this model in future for other data, the best way to do it is by creating a module which 
# consists of all ingredients shown here, but organized in classes. 